# More Realistic Problems
Where we left off, a greedy approach would have worked to solve that model. But problems typically have much more complicated business rules to follow -- which highlights the strengths of mathematical optimization. In this section, we'll use our handy **cheat sheet** to help model some of these scenarios.

## Pick up where we left off
When setting up the problem the first time we used `multidict` to create parts of our model. This time, let's read data in from a csv. 

In [1]:
%pip install gurobipy

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

### We have a helper function to solve and report values for our x decision variable
### If you download an run locally, you can get rid of the single line below
!wget -q https://raw.githubusercontent.com/yurchisin/ODSC2024/refs/heads/main/solve_and_report.py
from solve_and_report import *

### This time, we read in data from csv
on_hand = pd.read_csv('data_files/ing_amounts.csv', index_col=['ingredient']).squeeze()
chips_data = pd.read_csv('data_files/chips_data.csv', index_col=['chips']).squeeze()
recipes = pd.read_csv('data_files/recipes.csv', index_col=['chips', 'ingredients']).squeeze()

ingredients = on_hand.index.to_list()
chips = chips_data.index.to_list()

Note: you may need to restart the kernel to use updated packages.
/bin/bash: wget: command not found


## Original model

As a reminder:
- $x_c$ is the number of chips to make of denomination $c \in \{\$1, \$5, \$10, \$25, \$100, \$500, \$1000\}$.
- $v_c$ is the value associated with each chip.
- $q_i$ is the amount of ingredient $i \in \{\text{clay, lead, silver, gold}\}$ we have on hand.
- $r_{c,i}$ is the amount of **ingredient** $i$ used to make **chip** $c$. 

\begin{align*}
&\text{Maximize} \space \sum_c v_c*x_c \\
&\text{subject to:} \\
&\quad\sum_{c}r_{c, i} * x_{c} \le q_i, \space \text{for} \space i \in \{\text{clay, lead, silver, gold}\} \\
&\quad x_c \ge 0, \forall c \in \text{Chips}
\end{align*}

In [6]:
### Create a new model
model = gp.Model("Poker Chips")

### Decision variables
x = model.addVars(chips, vtype=GRB.INTEGER, name="chips")

### Ingredient constraint
ingredient_usage = model.addConstrs((gp.quicksum(x[c] * recipes[c, i] for c in chips) <= on_hand[i] for i in ingredients), name="ingredient_usage")

### Objective function
model.setObjective(gp.quicksum(x[i] * chips_data.value[i] for i in chips), sense=GRB.MAXIMIZE)

### Our helper function for this session:
solve_and_print_solution(model)

Optimal solution found
Objective value: 42143.0
chips[one]: 38.0
chips[five]: 1.0
chips[ten]: 0.0
chips[twenty-five]: 284.0
chips[one hundred]: 0.0
chips[five hundred]: 50.0
chips[thousand]: 10.0


## Decision expressions
Some times it is helpful to store quantities of interest to help make code easier to read, reduce clutter, or get key values quickly. For now, let's make an expression for the total number of chips made. 

Mathematically, let's first define the set and then the algebraic expression.  
- $C = \{\$1, \$5, \$10, \$25, \$100\, \$500, \$1000\}$

\begin{align*}
\text{total chips} &= \sum_{c \in C} x_c,
\end{align*}

In [21]:
total_chips  = gp.quicksum(x[c] for c in chips)
model.update()

In [ ]:
### Test your expressions here
total_chips.getValue()

## Changing our model

### New objective function
Instead of maximizing the total chip value, we are asked to maximize the total number of chips made. Write out and code this new objective.

\begin{equation*}
\text{Maximize} \space \sum_{c \in C} x_c
\end{equation*}

In [ ]:
### Set the objective to maximize the number of chips
model.setObjective(total_chips, GRB.MAXIMIZE)

solve_and_print_solution(model)

Note that we didn't have to do anything else other than re-run `setObjective`. 

### Meet minimum demand requirements
- View the chips data dataframe.
- Write constraints and code to meet a minimum demand for each chip value. 

In [ ]:
### Print each chip here
chips_data

Let $d_c$ be the demand for chip $c$.
\begin{equation*}
x_c \ge d_c, \forall c \in C
\end{equation*}

In [ ]:
### Ensure the production meets the minimal chip demand
meet_demand = model.addConstrs((x[c] >= chips_data.demand[c] for c in chips), name="meet_demand")

solve_and_print_solution(model)

### Cost of making chips:
- In the data frame above, we see there is a production cost per chip made. Write an expression for the total *variable cost* of chips made. 
- Add another constraint that limits the total variable cost to 80.
- Note that the demand constraints from before will still be active. 

Let $a_c$ be the variable cost of chip $c$. 
\begin{align*}
\text{total variable cost} = &\sum_c a_c*x_c  \\
\sum_c a_c*x_c \le 80
\end{align*}

In [ ]:
### Variable cost chip making cost
vCost_all_chips = sum(chips_data.cost[c] * x[c] for c in chips)
vBudget = model.addConstr(vCost_all_chips <= 80, name='v_budget')

solve_and_print_solution(model)

### Fixed costs: More decision variables
Our production machine can only make chips of one value at a time. When we want to switch, it takes time to setup with new materials, which adds a fixed cost to each chip value. Given this, we have to decide whether or not we want to make each type of chip. If we decide to make at least one \$10 chip, then we incur a cost of 30. 

Let's go step-by-step to add in the fixed costs for each chip and minimize the total costs, while meeting the above demand.
- First define a new set of variables indexed by chip type and call them as `y`.
- Write a constraint that links the number of chips made with the new binary variable (**HINT:** use the cheat sheet). 
- Don't resolve yet. 


Let $f_c$ be the fixed cost of chip $c$ and $y_c = 1$ if chip $c$ is made, $0$ otherwise. 

If we decide to make a \$10 chip, then $x_{ten} > 0$ and we want to have the associated cost go from 0 to 30. 

So the constraint is
\begin{equation*}
x_c \le M*y_c, \forall c \in C
\end{equation*}

But what do we choose for $M$?


In [27]:
y = model.addVars(chips, vtype=GRB.BINARY, name="make")

### Link binary variable to chip production using `big-M` constraints
M = 500
model.addConstrs((x[c] <= M * y[c] for c in chips), name="link_x_y")
model.update()

### Fixed costs: Combining costs
- Write an expression that finds the total fixed costs using the new variable and the `fixed_cost` column from `chips_data`.
- Add another constraint that limits the total cost to 280.
- Solve!

\begin{align*}
&\text{total fixed cost} = \sum_c f_c*y_c  \\
\text{total cost} = \space&\text{total fixed cost}  + \text{total variable cost} = \sum_c f_c*y_c + a_c*x_c 
\end{align*}

In [ ]:
fCost_all_chips = gp.quicksum(chips_data.fixed_cost[c] * y[c] for c in chips)
tCost_all_chips = vCost_all_chips + fCost_all_chips

tBudget = model.addConstr(tCost_all_chips <= 280, name='t_budget')

solve_and_print_solution(model)

Suppose due circumstances beyond our control, we can't produce all seven types of chips.  This means we can't meet demand as given, so let's remove that constraint.

In [29]:
model.remove(meet_demand)

### Using binary variables to control production

Specifically, these circumstances say we can produce at most five, but at least three types. Write two constraints using the new `y` decision variables to model that and then solve. 

In [ ]:
min_types = model.addConstr(y.sum() >= 3, name="min_types")
max_types = model.addConstr(y.sum() <= 5, name="max_types")

solve_and_print_solution(model)

# Extra Practice
It turns out the last request came from some folks who didn't believe mathematical optimization models can be changed so quickly to adjust to new scenarios and wanted to test us. Let's practice modeling additional scenarios so we're well prepared for anything. 

First, let's remind ourselves of initial model, but *maximizing total chips* made. 

In [41]:
### re-create a new model with max total chips made obj
model = gp.Model("Poker Chips")
x = model.addVars(chips, vtype=GRB.INTEGER, name="chips")
ingredient_usage = model.addConstrs((gp.quicksum(x[c] * recipes[c, i] for c in chips) <= on_hand[i] for i in ingredients), name="ingredient_usage")
total_chips  = x.sum()
model.setObjective(total_chips, GRB.MAXIMIZE)

## More decision expressions

Suppose we have grouped chip types into two categories: low and high value. High value chips are \$100, \$500, and \$1000, with the rest being low value. 

Define the sets and code the expressions as we did earlier.

Let's define more sets. 
- $H = \{\$100, \$500, \$1000\}$
- $L = C - H$

\begin{align*}
\text{value of total chips} &= \sum_{c \in C} v_c*x_c   \\
\text{value of high chips} &= \sum_{h \in H} v_h * x_h    \\
\text{value of low chips} &= \sum_{l \in L} v_l*x_l    \\
\end{align*}

In [42]:
high_chips = ['one hundred','five hundred', 'thousand']
low_chips = [c for c in chips if c not in high_chips]

### While we use `quicksum` here, thought there are more compact ways to write these. 
val_total_chips  = gp.quicksum(chips_data.value[c] * x[c] for c in chips)
val_high_chips = gp.quicksum(chips_data.value[c] * x[c] for c in high_chips)
val_low_chips  = gp.quicksum(chips_data.value[c] * x[c] for c in low_chips)

### Proportion of total chips

#### Formulate and code the following constraints
- The value produced by high value chips cannot exceed the 50% of the value produced by low value chips.
- Each of the of high value chips must be at least 5% of the total chips made and each of the low value chips cannot be more than 20%. 
- The \$25 chip is our most used; make sure that it has the maximum number among all chips.
- Limit any single chip type to 30% of total chips produced.

\begin{equation*}
\sum_{h \in H} v_h*x_h \le 0.5*\sum_{l \in L} v_l*x_l
\end{equation*}

In [ ]:
### The value produced by high value chips cannot exceed the 50% of the value produced by low value chips
value_balance = model.addConstr(val_high_chips <= .5*val_low_chips, name='value_balance')

solve_and_print_solution(model)

We stored each constraint as an object, which makes it easier to interact with these later. Now, let's remove the `value_balance` constraint. 

In [44]:
model.remove(value_balance)

\begin{align*}
x_{l} \le& 0.20*\sum_{c \in C} x_c, \forall l\in L \\
x_{h} \ge& 0.05*\sum_{c \in C} x_c, \forall h\in H \\
\end{align*}

In [ ]:
low_chip_ub = model.addConstrs((x[l] <= 0.20 * total_chips for l in low_chips), name='low_chip_ub')
high_chip_lb = model.addConstrs((x[h] >= 0.05 * total_chips for h in high_chips), name='high_chip_lb')

solve_and_print_solution(model)

\begin{align*}
x_{twenty-five} \ge x_{c}, \forall c \in C-\{\$25\}.
\end{align*}

In [ ]:
### Use this set since this constraint doesn't need to apply to $25
C_minus25 = [cc for cc in chips if cc != 'twenty-five']

max_25 = model.addConstrs((x['twenty-five'] >= x[c] for c in C_minus25), name='max_25')

solve_and_print_solution(model)

\begin{align*}
x_{c} \le 0.3*\sum_cx_{c}, \forall c \in C
\end{align*}

In [ ]:
percent30 = model.addConstrs((x[c] <= 0.3*total_chips for c in chips), name='percent30')

### Let's check in on our model and remove some constraints
As you code a model, you may want to make sure adding variables, constraints, and the objective all go as planned.

In [ ]:
### Print the model object to get a quick summary
print(model)

### Writing a *.lp file is a great way to get a look at your model
model.write('our_model.lp')

The `Slack` of a constraint is the gap between the left-hand side and right-hand side values at the optimal solution. Printing this for the `ingredient_usage` constraints will show how much of each material is remaining. 

In [ ]:
for i in ingredients:
    print(f"Remaining {i}: {ingredient_usage[i].Slack}")

In [50]:
model.remove([low_chip_ub, high_chip_lb, max_25])
model.update()

In [ ]:
model.getConstrs()

### Logical constraints with binary variables
Now that we have binary decision variables that show which chip types are made, we can model logical relationships between them. Model the following statements and write `gurobipy` code. 

We won't solve a model with these. 
- We **can** make either \$500 **or** \$1000 chips, and **possibly both** (at least one).
- We **must** make either \$500 **or** \$1000 chips, but **not both** (exactly one).
- We make between 2 and 3 types of low value. 

In [52]:
### add our binary decision variables and corresponding logic. 
y = model.addVars(chips, vtype=GRB.BINARY, name="make")
M = 500
model.addConstrs((x[c] <= M * y[c] for c in chips), name="link_x_y")
model.update()

- We **can** make either \$500 **or** \$1000 chips, and **possibly both** (at least one).
\begin{equation*}
y_{five\space hundred} + y_{thousand} \le 1 
\end{equation*}

- We **must** make either \$500 **or** \$1000 chips, but **not both** (exactly one).
\begin{equation*}
y_{five\space hundred} + y_{thousand} = 1 
\end{equation*}

- We make between 2 and 3 types of low value. 
\begin{equation*}
2 \le \sum_l y_l \le 3
\end{equation*}

In [ ]:
# We can make either $500 or $1000 chips, but not both (at most one)
model.addConstr(y['five hundred'] + y['thousand'] <= 1, name="at_most_one_high_value")

# We must make either $500 or $1000 chips, but not both (exactly one)
model.addConstr(y['five hundred'] + y['thousand'] == 1, name="exactly_one_high_value")

# We make between 2 and 3 types of low value and exactly one of the high value chips
model.addConstr(gp.quicksum(y[l] for l in low_chips) >= 2, name="min_low_value_types")
model.addConstr(gp.quicksum(y[l] for l in low_chips) <= 3, name="max_low_value_types")

### Another shortcut using the gurobipy API
model.addConstr(gp.quicksum(y[l] for l in low_chips) == [2,3], name="max_low_value_types")


### Conditional statements
- If we make \$1 chips, then we **must** make \$5 chips.
- If we make \$1 chips, then we **must** make \$5 **and** \$10 chips.
- If we make \$1 chips, then we **must** make \$5 **or** \$10 chips (or both). 

- If we make \$1 chips, then we **must** make \$5 chips.
\begin{align*}
y_{one} &\le y_{five} \quad\quad\quad\space
\end{align*}

- If we make \$1 chips, then we **must** make \$5 **and** \$10 chips.

\begin{align*}
y_{one} &\le y_{five}, \\
y_{one} &\le y_{ten} \\
&\text{or} \\
2*y_{one} &\le y_{five} + y_{ten}
\end{align*}

- If we make \$1 chips, then we **must** make \$5 **or** \$10 chips (or both). 

\begin{align*}
\quad y_{one} &\le y_{five} + y_{ten}
\end{align*}


In [ ]:
#### Logistical constraint candidates:

### If we make $1 chips, then we must make $5 chips
model.addConstr(y['one'] <= y['five'], name="one_implies_five")

### If we make $1 chips, then we must make $5 and $10 chips
model.addConstr(y['one'] <= y['five'], name="one_implies_five")
model.addConstr(y['one'] <= y['ten'], name="one_implies_ten")

model.addConstr(2*y['one'] <= y['five'] + y['ten'], name="one_implies_ten")

### If we make $1 chips, then we must make $5 or $10 chips
model.addConstr(y['one'] <= y['five'] + y['ten'], name="one_implies_ten")

## Semi-continuous and semi-integer variables

### Using 'big M'
- For each high value chip, if we make them, then we need to make between 5 and 30 (inclusive)
- For each lower value chip, if we make them, then we need to make at least 100


\begin{align*}
5*y_h &\le x_h \le 30*y_h,\forall h \in H  \\
50*y_l &\le x_l \le M*y_l,\forall l \in L
\end{align*}

In [ ]:
#### Using big M
### Constraint: For each high value chip, if we make them, then we need to make between 5 and 20 (inclusive)
min_high_value = model.addConstrs((x[c] >= 2 * y[c] for c in high_chips), name="min_high_value")
max_high_value = model.addConstrs((x[c] <= 30 * y[c] for c in high_chips), name="max_high_value")

### Constraint: For each low value chip, if we make them, then we need to make at least 100
min_low_value = model.addConstrs((x[c] >= 30 * y[c] for c in low_chips), name="min_low_value")
min_low_value = model.addConstrs((x[c] <= M * y[c] for c in low_chips), name="min_low_value")

### Using gurobipy semi-integer variable 
We won't run this, but we see how we can add these variables

In [ ]:
x = {}
for c in high_chips:
    x[c] = model.addVar(vtype=GRB.SEMIINT, lb=5, ub=30, name=f"chips_{c}")

for c in low_chips:
    x[c] = model.addVar(vtype=GRB.SEMIINT, lb=100, ub=M, name=f"chips_{c}")

## Multi-objective optimization
- We are asked to maximize the total chip value made while minimizing costs. 
- Math optimization cannot **simultaneously** work on two objectives -- there's always a tradeoff. 
- Two types on multi-objective: hierarchical, and weighted (blended). Consult your **cheat sheet** for more. 
- After talking more about how we want to prioritize the objectives, we want to:
    - First maximize the **total number of chips** made. 
    - Then maximize the total value of chips produced while not decreasing the number of chips made by more than 10%
- Also include the following constraints we already discussed:
    - Ingredient limits
    - Limit any single chip type to 30% of total
    - Overall cost must be less than 280

### DIY hierarchical multi-objective optimzation
- Solve the a model that maximizes the total number of chips made.
- Add a new constraint that uses this objective value as a new bound on this decision expression. Let $z$ be this value.
\begin{equation*}
\sum_c x_c \ge 0.9*z
\end{equation*}
- Set the new objective to minimize total cost, as before. 
- Solve!

In [21]:
model = gp.Model("Poker Chips")

M = 500

### Decision variables
x = model.addVars(chips, vtype=GRB.INTEGER, name="chips")
y = model.addVars(chips, vtype=GRB.BINARY, name="make")

### Other sets and expressions
high_chips = ['one hundred','five hundred', 'thousand']
low_chips = [c for c in chips if c not in high_chips]

total_chips = x.sum()
val_total_chips  = gp.quicksum(chips_data.value[c] * x[c] for c in chips)
val_high_chips = gp.quicksum(chips_data.value[c] * x[c] for c in high_chips)
val_low_chips  = gp.quicksum(chips_data.value[c] * x[c] for c in low_chips)
vCost_all_chips = gp.quicksum(chips_data.cost[c] * x[c] for c in chips)
fCost_all_chips = gp.quicksum(chips_data.fixed_cost[c] * y[c] for c in chips)
tCost_all_chips = vCost_all_chips + fCost_all_chips

### Constraints
ingredient_usage = model.addConstrs((gp.quicksum(x[c] * recipes[c, i] for c in chips) <= on_hand[i] for i in ingredients), name="ingredient_usage")
#max_25 = model.addConstrs((x['twenty-five'] >= x[c] for c in [cc for cc in chips if cc != 'twenty-five']), name='max_25')
percent30 = model.addConstrs((x[c] <= 0.3*total_chips for c in chips), name='percent30')
link_x_y = model.addConstrs((x[c] <= M * y[c] for c in chips), name="link_x_y")
tBudget = model.addConstr(tCost_all_chips <= 220, name='t_budget')

### First Objective
model.setObjective(total_chips, sense=GRB.MAXIMIZE)

solve_and_print_solution(model)

Optimal solution found
Objective value: 367.0
chips[one]: 0.0
chips[five]: 47.0
chips[ten]: 110.0
chips[twenty-five]: 110.0
chips[one hundred]: 100.0
chips[five hundred]: 0.0
chips[thousand]: 0.0
make[one]: 1.0
make[five]: 1.0
make[ten]: 1.0
make[twenty-five]: 1.0
make[one hundred]: 1.0
make[five hundred]: 0.0
make[thousand]: 0.0


In [18]:
z = model.ObjVal
print(z)
multi_obj = model.addConstr(total_chips >= 0.9*z)
model.setObjective(val_total_chips, sense=GRB.MAXIMIZE)
solve_and_print_solution(model)

367.0
Optimal solution found
Objective value: 29050.0
chips[one]: 0.0
chips[five]: 89.0
chips[ten]: 103.0
chips[twenty-five]: 103.0
chips[one hundred]: 0.0
chips[five hundred]: 50.0
chips[thousand]: 0.0
make[one]: 0.0
make[five]: 1.0
make[ten]: 1.0
make[twenty-five]: 1.0
make[one hundred]: 0.0
make[five hundred]: 1.0
make[thousand]: 0.0


#### Homework!
Use the **cheat sheet** to do this using the `gurobipy` multi-objective functionality.

In [ ]:
model.dispose()